# Dhruv Rana #

In [23]:
import glob
import numpy as np
import tensorflow as tf
from keras.utils import load_img, img_to_array
from keras.applications.resnet import ResNet50
from keras.applications.resnet import preprocess_input, decode_predictions

model = ResNet50(weights='imagenet')

true_label = ['vending machine', 'vending machine' , 'golf cart', 'golf cart']

# sorted alphabatically
images = glob.glob('data\\*.jpg')
images = sorted(images)

for filename in images :
    img = load_img(filename, target_size=(224, 224))
    img_arr = img_to_array(img)
    img_batch = np.expand_dims(img_arr, axis=0)
    img_processed = preprocess_input(img_batch)

    prediction = decode_predictions(model.predict(img_processed),top=1)
    print('Predicted:', prediction)

C:\Users\dhruv\AppData\Roaming\Python\Python39\site-packages\keras\engine\training_v1.py:2356: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  updates=self.state_updates,


Predicted: [[('n03445924', 'golfcart', 0.9907712)]]
Predicted: [[('n03445924', 'golfcart', 1.0)]]
Predicted: [[('n04525305', 'vending_machine', 0.99944645)]]
Predicted: [[('n04525305', 'vending_machine', 0.9989009)]]
